In [46]:
from bs4 import BeautifulSoup
import requests

host = "https://refactoring.guru/design-patterns/"

page = requests.get(host + "catalog/")
print(page.status_code)

200


In [47]:
soup = BeautifulSoup(page.content, 'html.parser')

patterns = soup.select(".pattern-card")
pattern_names = [p.text for pattern in patterns for p in pattern.select(".pattern-name")]
pattern_names = set(pattern_names)
pattern_names

{'Abstract Factory',
 'Adapter',
 'Bridge',
 'Builder',
 'Chain of Responsibility',
 'Command',
 'Composite',
 'Decorator',
 'Facade',
 'Factory Method',
 'Flyweight',
 'Iterator',
 'Mediator',
 'Memento',
 'Observer',
 'Prototype',
 'Proxy',
 'Singleton',
 'State',
 'Strategy',
 'Template Method',
 'Visitor'}

In [48]:
# because every pattern pages link suffixed by modified pattern name with changing whitespace to '-'
# we can generate link to every patterns
pattern_links = [host + p.lower().replace(" ", "-") for p in pattern_names]

# access all the data
patterns_pages = [requests.get(link) for link in pattern_links]


In [60]:
pattern_soups = [BeautifulSoup(p.content, 'html.parser') for p in patterns_pages]

# data we want to retrieve
# - every part of the page design pattern structure
# - for part that has specific steps or list, parse it again
# - for part that emphasize specific keyword, safe it
# - for relations with other design pattern we can create the connection and the text
# - for any part, save the text with BIGTEXT
# - for any part, save the pictures
# - preprocess each paragraph with https://rapidapi.com/MeaningCloud/api/summarization

In [109]:
# see if each segments are the same
names_pages = zip(pattern_names, pattern_soups)

segment = set()
for name, page in names_pages:
  segments = [s.text for s in page.select("article > .section > h2")]
  print(name, ": ", ", ".join(segments), "\n\n")
  if (segment == set()):
    segment = set(segments)
  elif segment != set(segments):
    print("segment differ ", segment.symmetric_difference(set(segments)),  "\n")

# Additional segment are extra content and real world analogy in Abstract Factory and Factory Method

Composite :   Intent,  Problem,  Solution,  Real-World Analogy,  Structure,  Pseudocode,  Applicability,  How to Implement,  Pros and Cons,  Relations with Other Patterns,  Code Examples 


Chain of Responsibility :   Intent,  Problem,  Solution,  Real-World Analogy,  Structure,  Pseudocode,  Applicability,  How to Implement,  Pros and Cons,  Relations with Other Patterns,  Code Examples 


Iterator :   Intent,  Problem,  Solution,  Real-World Analogy,  Structure,  Pseudocode,  Applicability,  How to Implement,  Pros and Cons,  Relations with Other Patterns,  Code Examples 


Abstract Factory :   Intent,  Problem,  Solution,  Structure,  Pseudocode,  Applicability,  How to Implement,  Pros and Cons,  Relations with Other Patterns,  Code Examples,  Extra Content 


segment differ  {' Extra Content', ' Real-World Analogy'} 

Template Method :   Intent,  Problem,  Solution,  Real-World Analogy,  Structure,  Pseudocode,  Applicability,  How to Implement,  Pros and Cons,  Relations with Oth